# Clustering Proteins and Data Integration
# 1. Introduction

# 2. Preparing and Cleaning the Data

In [1]:
'''!pip install kneed'''

'!pip install kneed'

In [2]:
#import os
import sys
from pathlib import Path
# Data processing and analysis
import pandas as pd
import numpy as np
import openpyxl
import seaborn as sns
import networkx as nx
import community as community_louvain
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
# Standard library imports
import ast
import subprocess
import logging
import time
from datetime import datetime
import shutil
from io import StringIO
import re
from scipy.cluster.hierarchy import dendrogram
from sklearn.metrics import silhouette_score
from kneed import KneeLocator
import matplotlib.cm as cm
%matplotlib inline
from matplotlib.colors import to_rgba, LinearSegmentedColormap
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import ipywidgets as widgets
from IPython.display import display
import kaleido

# Machine learning and statistical analysis
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.decomposition import PCA, NMF
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.cluster.hierarchy import linkage, fcluster
import umap
import scipy
from scipy import stats
from scipy.cluster.hierarchy import linkage, dendrogram
import scipy.cluster.hierarchy as sch
from statsmodels.stats.multitest import multipletests
from scipy.spatial.distance import pdist
from scipy.stats import spearmanr, kruskal, mannwhitneyu
from scipy.signal import savgol_filter
from sklearn.ensemble import RandomForestClassifier
import shap
from sklearn.preprocessing import MinMaxScaler

from joblib import Parallel, delayed
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.cluster.hierarchy import linkage, fcluster
# Utility libraries
import gzip
import random
from natsort import natsorted
from typing import Dict, List, Tuple, Set, Optional
import gc
import joblib
import os
import json
import pyarrow.parquet as pq

os.environ['DISPLAY'] = ':0'

# Dash
import dash
from dash import dcc, html, Input, Output, callback, State
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import umap
# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s')
# Own Scoring system
import corrosion_scoring as cs


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.12/runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.12/runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "/home/beatriz/MIC/3_combined/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/beatriz/MIC/3_combined/.venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/beatriz

In [3]:
# Determine the environment
if "google.colab" in sys.modules:
    print("Running in Google Colab environment")
    # for colab
    base_dir = Path("/content/drive/MyDrive/MIC")
    abundance_excel = base_dir / "data_picrust/merged_to_sequence.xlsx"
    output_large = base_dir / "output_large"
    output_base = base_dir
    market_dir = base_dir / "output_large" 
    #Directory to keep some Results
    large_dir = base_dir / "2_Micro/data_visual"
    large_dir.mkdir(parents=True, exist_ok=True)

elif Path("/kaggle").exists():
    print("Running in Kaggle environment")
    # For Kaggle work# Input datasets (read-only in Kaggle) 
    base_dir = Path("/kaggle/input/")  
    abundance_excel = base_dir / "new-picrust/merged_to_sequence.xlsx" 
    #Input physicochemical variables
    data_physicochemical = base_dir / "physicochemical-parameters/Physicochemical.xlsx"
    combined_input = base_dir  / "combined_markers.xlsx"
    #===============================================
    #Directory to keep  Results
    output_base = Path("/kaggle/working/")
    shared_dir = output_base/"Visualisations"
    shared_dir.mkdir(parents=True, exist_ok=True)
    combined_path = output_base / "combined_markers.xlsx"
      
else:
    print("Running in local (VSCode) environment")
    base_dir = Path("data")
    base_dir.mkdir(parents=True, exist_ok=True)
    # Base Paths for local environment
    abundance_excel = base_dir / "merged_to_sequence.xlsx"
    #Input physicochemical variables8ikk                           ´
    data_physicochemical = Path("/home/beatriz/MIC/1_Physicochemical/Data/Physicochemical.xlsx")
    #================================================
    # This files are too large for github and are store on Kaggle for educational purposes
    output_large = Path("/home/beatriz/MIC/output_large")
    output_base = base_dir 
    #Directory to keep some Results
    shared_dir= Path("/home/beatriz/SharedFolder/Visualisations/")
    combined_path = base_dir / "combined_markers.xlsx"
       

Running in local (VSCode) environment


### Importing the files

In [4]:
# physicochemical data data from notebook 8 repo 1_Physicochemical
all_physichem = pd.read_excel(data_physicochemical, sheet_name='all_physicochemical', engine ='openpyxl')
# Microbiological Data from notebook 6 repo 2_Micro.
df_proteins= pd.read_excel(combined_path, sheet_name='df_proteins',  engine ='openpyxl')
#mapping of genera to site from the original df for validation
genus_site_df = pd.read_excel(combined_path, sheet_name='genus_to_sites', engine ='openpyxl')
# All markers after pattern, integration, classification, increasing, and balance functions
balanced_markers= pd.read_excel(combined_path, sheet_name='balanced_markers', engine ='openpyxl')

# 3 Functional Category groups and Random Forest importance
Using functional Category groups and Random Forest importance to select top markers.This rf integrates the functional categories from the cs package, uses Random Forest to identify the most important features for predicting corrosion risk and groups them by functional category to understand which biological processes are most informative, but also an additional step will communicate the niche specific processes. Then uses SHAP values for more interpretable feature importance. At end a selected balanced set of features representing different functional categories is chosen mapping the features back to specific proteins for biological interpretation.
The chosen column to be associating the data or to serve as integration factor was the "functional_categories" or "fc_present" both of which presented similar entries and it was finally decided to cluster instead according to the "niche_specific_pathways" the reason is because the granularity it contains more differentiation between proteins already very similar all being at this point mostly on category 3. The functional_category, corrosion_mechanisms, enzyme_class and explanation columns are also brough forward to have the richness of the information. Additionally Columns Sites, Categories, combined_score, protein_name and genus would be also brought. 

The results from the function analyze_protein_hierachy gives a comprehensive dataframe. This is a single flattened DataFrame combining both descriptive and explanatory metadata and numerical attributes those are:  
* Fields that support interpretation and discussion of each protein:  
 ['Sites', 'protein_name', 'genera_count', 'genera', 'functional_categories', 'niche_pathways', 'enzyme_class', 'corrosion_mechanisms',  'explanation', 'combined_score']
* Fields that with values that feed a supervised neural network in Notebook 2:  
['Sites','Category','protein_name', norm_abund_contri', 'fold_change_2vs1', 'log2fc_2vs1', 'fold_change_3vs2', 'log2fc_3vs2', 'fold_change_3vs1', 'log2fc_3vs1',  'specificity', 'prevalence', 'max_abs_log2fc', 'combined_score'] 
The idea of the operation is to mantain the original relationships so hat each row represents one unique (protein-genus-site) observation preserving the original abundance granularity from the PICRUSTt2  OTU→EC→protein mapping.

In [11]:
def analyze_protein_hierarchy_rf(protein_df, n_estimators=100, n_top_proteins=50, random_state=42):
    """
    Analyzes protein hierarchy using Random Forest for feature importance,
    performs hierarchical clustering on niche-specific pathways,
    and returns a comprehensive DataFrame with both descriptive and numerical attributes.
    
    Parameters: protein_df : DataFrame Input DataFrame containing protein data with all required columns
    n_estimators : int, default=100   Number of trees in the Random Forest
    n_top_proteins : int, default=50  Number of top proteins to select based on importance
    random_state : int, default=42  Random seed for reproducibility
    
    Returns:  DataFrame  Comprehensive DataFrame with selected proteins and all required attributes
    """
    # Step 1: Prepare data for Random Forest: features for importance ranking (numerical columns)
    numerical_features = ['fold_change_2vs1', 'log2fc_2vs1', 'fold_change_3vs2', 'log2fc_3vs2', 'fold_change_3vs1', 'log2fc_3vs1', 
                         'specificity', 'prevalence', 'max_abs_log2fc', 'combined_score']
    
    # Create dummy target for feature importance using 'Category' as a proxy target
    X = protein_df[numerical_features].fillna(0)
    y = protein_df['Category']
    
    # Step 2: Train Random Forest for feature importance    
    rf = RandomForestClassifier(n_estimators=n_estimators, random_state=random_state)
    rf.fit(X, y)
    
    # Step 3: Calculate SHAP values for feature importance interpretation
    explainer = shap.TreeExplainer(rf)
    shap_values = explainer.shap_values(X)

    # Average across samples
    sample_averaged = np.mean(shap_values, axis=0)  # Shape: (10, 3)

    # Take absolute values
    absolute_values = np.abs(sample_averaged)  # Still (10, 3)

    # Average across classes to get a single importance score per feature
    shap_importance = np.mean(absolute_values, axis=1)  # Final shape: (10,)

    # Create feature importance DataFrame
    feature_importance = pd.DataFrame({
        'feature': numerical_features,
        'importance': rf.feature_importances_,
        'shap_importance': shap_importance
    })
    feature_importance = feature_importance.sort_values('shap_importance', ascending=False)
    #======================================================
    # Step 4: Hierarchical clustering based on niche-specific pathways
    # First, create a distance matrix from the pathway information: Convert niche-specific pathways to a TF-IDF matrix
    tfidf = TfidfVectorizer()
    # Handle missing values
    pathway_texts = protein_df['niche_specific_pathways'].fillna('')
    pathway_matrix = tfidf.fit_transform(pathway_texts)
    
    # Compute hierarchical clustering
    Z = linkage(pathway_matrix.toarray(), method='ward')
    
    # Determine optimal number of clusters compute different clustering solutions
    max_clusters = min(20, len(protein_df))
    silhouette_scores = []
    
    for n_clusters in range(2, max_clusters + 1):
        clusters = fcluster(Z, n_clusters, criterion='maxclust')
        if len(np.unique(clusters)) <= 1:
            silhouette_scores.append(0)
        else:
            silhouette_scores.append(silhouette_score(pathway_matrix.toarray(), clusters))
    
    # Find optimal number of clusters using knee point detection
    kl = KneeLocator(range(2, max_clusters + 1), silhouette_scores, 
                    curve='concave', direction='increasing')
    optimal_clusters = kl.elbow if kl.elbow else 5  # Default to 5 if no clear elbow
    
    # Assign clusters
    clusters = fcluster(Z, optimal_clusters, criterion='maxclust')
    protein_df['pathway_cluster'] = clusters
    
    # Step 5: Evaluate protein importance within functional categories
    # Add importance to original dataframe
    protein_importance = {}
    for idx, row in protein_df.iterrows():
        protein_name = row['protein_name']
        protein_importance[protein_name] = protein_importance.get(protein_name, 0)
        
        # Calculate importance based on the SHAP values of numerical features
        for i, feature in enumerate(numerical_features):
            if not pd.isna(row[feature]):
                feature_weight = feature_importance.loc[
                    feature_importance['feature'] == feature, 'shap_importance'].values[0]
                protein_importance[protein_name] += row[feature] * feature_weight
    
    # Create a DataFrame with protein importance
    protein_importance_df = pd.DataFrame({
        'protein_name': list(protein_importance.keys()),
        'importance_score': list(protein_importance.values())
    }).sort_values('importance_score', ascending=False)
    
    # Step 6: Select a balanced set of top proteins. First, group by functional categories
    fc_grouped = protein_df.groupby('functional_categories_present')
    
    # Select top proteins from each functional category
    selected_proteins = []
    for fc, group in fc_grouped:
        # Get proteins in this functional category
        fc_proteins = group['protein_name'].unique()
        
        # Get importance scores for these proteins
        fc_importance = protein_importance_df[
            protein_importance_df['protein_name'].isin(fc_proteins)
        ]
        
        # Select top proteins from this category (proportional to category size)
        n_select = max(1, int(len(fc_proteins) * n_top_proteins / len(protein_df['protein_name'].unique())))
        top_fc_proteins = fc_importance.head(n_select)['protein_name'].tolist()
        selected_proteins.extend(top_fc_proteins)
    
    # Step 7: Create and return the comprehensive DataFrame and filter to selected proteins
    comprehensive_df = protein_df[protein_df['protein_name'].isin(selected_proteins)]
    
    # Add group count information
    genera_counts = comprehensive_df.groupby('protein_name')['Genus'].nunique().reset_index()
    genera_counts.columns = ['protein_name', 'genera_count']
    
    genera_lists = comprehensive_df.groupby('protein_name')['Genus'].apply(list).reset_index()
    genera_lists.columns = ['protein_name', 'genera']
    
    # Merge with comprehensive_df
    comprehensive_df = comprehensive_df.merge(genera_counts, on='protein_name', how='left')
    comprehensive_df = comprehensive_df.merge(genera_lists, on='protein_name', how='left')
    
    # Add importance scores
    comprehensive_df = comprehensive_df.merge(
        protein_importance_df[['protein_name', 'importance_score']], 
        on='protein_name', how='left'
    )
    
    # Rename columns for clarity
    comprehensive_df = comprehensive_df.rename(columns={
        'niche_specific_pathways': 'niche_pathways'
    })
    
    # Ensure all required columns are present
    descriptive_fields = ['Sites', 'protein_name', 'genera_count', 'genera', 
                         'functional_categories', 'niche_pathways', 'enzyme_class', 
                         'corrosion_mechanisms', 'explanation', 'combined_score']
    
    numerical_fields = ['Sites', 'Category', 'protein_name', 'norm_abund_contri',
                       'fold_change_2vs1', 'log2fc_2vs1', 'fold_change_3vs2', 
                       'log2fc_3vs2', 'fold_change_3vs1', 'log2fc_3vs1', 
                       'specificity', 'prevalence', 'max_abs_log2fc', 'combined_score']
    
    # Select columns for final output
    all_required_fields = list(set(descriptive_fields + numerical_fields))
    output_columns = [col for col in all_required_fields if col in comprehensive_df.columns]
    
    # Add importance score and cluster information
    output_columns += ['importance_score', 'pathway_cluster']
    
    return comprehensive_df[output_columns]

protein_hierarchy = analyze_protein_hierarchy_rf(df_proteins, n_estimators=100, n_top_proteins=50, random_state=42)

The protein_hierarchy dataframe seem a good mixture of numerical and descriptive parameters, however the genera count appear no to reflect that the protein corresponds to a substantial genera prevalence, due to that, genera count sort the genera.

In [12]:
protein_hierarchy= protein_hierarchy.sort_values('genera_count', ascending=False).head(200).copy()
protein_hierarchy["protein_name"].unique()

array(['enoyl-[ reductase [ (ec 1.3.1.9)',
       '3-oxoacyl-[ reductase (ec 1.1.1.100)',
       'enoyl-[ reductase (nadph, si-specific); acyl-acp-dehydrogenase',
       'glutathione hydrolase proenzyme (ec 2.3.2.2) 3.4.19',
       'gmp-synthase [ (ec 6.3.5.2) (gmp-synthetase) (glutamine',
       'glutamine-dependent nad(+) synthetase (ec 6.3.5.1)',
       'aspartate 1-decarboxylase (ec 4.1.1.11) (aspartate',
       'phosphoenolpyruvate carboxykinase [ (pep carboxykinase',
       'aspartate transaminase; glutamic-oxaloacetic transaminase;',
       'dna ligase (ec 6.5.1.2) (polydeoxyribonucleotide-synthase',
       "holo-[ synthase (holo-acp-synthase) (ec 2.7.8.7) (4'",
       'enoyl-[ reductase (nadh); enoyl-[ reductase; enoyl',
       'bifunctional enzyme ispd-ispf [',
       'ferredoxin---nadp+ reductase; ferredoxin-nicotinamide'],
      dtype=object)

## Data for Descriptive Table
In this table the protein_names would be agrupated so that all the other descriptives would be aggregated to visualise the whole data. On the other hand the dataframe with the whole result protein_hierarchi which was ordered by genera count is brough to the next step namely the pca to visualise the natural patterns of the data.

In [14]:
# First, establish the descriptors and categorical descriptors
categorical_descriptors = ['protein_name', 'genera', 'functional_categories', 'niche_pathways', 
                         'enzyme_class', 'corrosion_mechanisms', 'explanation', 'Sites']
numerical_descriptors = ['genera_count', 'combined_score']

# 1. Create the protein_descriptors dataframe
# Start with the columns we need
protein_descriptors_raw = protein_hierarchy[categorical_descriptors + numerical_descriptors]

# Define custom aggregation functions for different column types
def agg_lists(x):
    # If already a list, flatten and get unique items
    if isinstance(x.iloc[0], list):
        flattened = []
        for item in x:
            if isinstance(item, list):
                flattened.extend(item)
            else:
                flattened.append(item)
        # Convert items to strings to make them hashable, then back to list
        return list(set([str(i) for i in flattened]))
    # If not a list but we want a list of unique values
    else:
        return list(set([str(i) for i in x]))

def agg_first(x):
    return x.iloc[0]

# Create aggregation dictionary
agg_dict = {
    'combined_score': 'mean',
    'genera_count': 'sum' 
}

# Specify aggregation method for each descriptor column
for col in categorical_descriptors:
    if col != 'protein_name':  # Skip the groupby column
        if col in ['genera', 'Sites']:  # Columns that should be lists of unique values
            agg_dict[col] = agg_lists
        else:  # Columns where we just want the first value
            agg_dict[col] = agg_first

# Group by protein_name and aggregate
protein_descriptors = protein_descriptors_raw.groupby('protein_name').agg(agg_dict)
protein_descriptors = protein_descriptors.sort_values(by='genera_count', ascending=False).head(100)

print(protein_descriptors.head(20))


                                                    combined_score  \
protein_name                                                         
enoyl-[ reductase [ (ec 1.3.1.9)                         53.668974   
3-oxoacyl-[ reductase (ec 1.1.1.100)                     53.655383   
enoyl-[ reductase (nadph, si-specific); acyl-ac...       53.143900   
glutathione hydrolase proenzyme (ec 2.3.2.2) 3....       53.203451   
gmp-synthase [ (ec 6.3.5.2) (gmp-synthetase) (g...       52.725787   
glutamine-dependent nad(+) synthetase (ec 6.3.5.1)       52.708479   
aspartate 1-decarboxylase (ec 4.1.1.11) (aspartate       52.442570   
aspartate transaminase; glutamic-oxaloacetic tr...       53.070554   
phosphoenolpyruvate carboxykinase [ (pep carbox...       52.688025   
dna ligase (ec 6.5.1.2) (polydeoxyribonucleotid...       52.123479   
enoyl-[ reductase (nadh); enoyl-[ reductase; enoyl       53.549363   
holo-[ synthase (holo-acp-synthase) (ec 2.7.8.7...       52.633479   
bifunctional enzyme 

# Integration with Data to equilibrate Class Distribution

In [9]:

# Get all sites from classified_results
all_classified_sites = results_dict['classified_results']['Sites'].unique()
print(f"Total sites in classified_results: {len(all_classified_sites)}")

# Get sites from balanced_markers
balanced_sites = results_dict['balanced_markers']['Sites'].unique()
print(f"Sites in balanced_markers: {len(balanced_sites)}")

# Find missing sites
missing_sites = set(all_classified_sites) - set(balanced_sites)
print(f"Missing sites: {len(missing_sites)}")

# For each missing site, get its data from classified_results
missing_data = results_dict['classified_results'][
    results_dict['classified_results']['Sites'].isin(missing_sites)
]

# You can either:
# 1. Keep this as a separate dataset for analysis
results_dict['missing_sites_data'] = missing_data

# 2. Or combine with balanced_markers for a complete dataset
complete_dataset = pd.concat([
    results_dict['balanced_markers'], 
    missing_data
], ignore_index=True)
results_dict['complete_dataset'] = complete_dataset

# Check if all sites are now accounted for
all_sites_in_complete = results_dict['complete_dataset']['Sites'].unique()
print(f"Sites in complete dataset: {len(all_sites_in_complete)}")

NameError: name 'results_dict' is not defined

In [ ]:
# 2. Create the protein_nn dataframe
protein_nn_columns = ['Sites', 'Category', 'protein_name', 'norm_abund_contri',
                     'fold_change_2vs1', 'log2fc_2vs1', 'fold_change_3vs2',
                     'log2fc_3vs2', 'fold_change_3vs1', 'log2fc_3vs1',
                     'specificity', 'prevalence', 'max_abs_log2fc', 'combined_score']

protein_nn_raw = protein_hierarchy[protein_nn_columns].head(100).copy()

# Since we need Sites as the index and protein_names as columns, with norm_abund_contri as values, we'll pivot the dataframe
protein_nn_pivoted = protein_nn_raw.pivot_table(
    index='Sites',
    columns='protein_name',
    values='norm_abund_contri',
    aggfunc='mean'  
)

# preserving category
site_categories = protein_nn_raw[['Sites', 'Category']].drop_duplicates().set_index('Sites')

# Merge the Category back into the pivoted dataframe
protein_nn = protein_nn_pivoted.copy()
protein_nn = protein_nn.join(site_categories)

# Move Category to the beginning of the dataframe
category_col = protein_nn.pop('Category')
protein_nn.insert(0, 'Category', category_col)

In [ ]:
len(protein_hierarchy["Sites"].unique())

15

# 4. Novel Candidates
During the statistical feature analysis in Notebook 3, several bacterial genera were identified that showed strong correlations with the high-risk corrosion failure category. This led to hypothesize the potential of  of this microorganisms to be associated with corrosion processes and which were not previously documented.
In Notebook 4, we conducted a comprehensive literature review using academic databases to investigate the bacterial genera. The findings confirmed that these microorganisms have no prior documentation in scientific literature linking them to corrosion-related processes.
Now, we aim to deploy the analyze_protein_hierarchy function to examine whether these bacteria express proteins that are known to induce or accelerate corrosion. This analysis will help determine if these genera possess the molecular machinery to contribute to corrosion despite their absence from corrosion-related literature, potentially identifying novel microbial contributors to corrosion processes that have been overlooked in previous research.

In [ ]:
# Creating a dictionary of genera identified as nobel on Notebook 4.
new_genera= ['Oxalobacteraceae_unclassified', 'Oxobacter', 'Mycoplana', 'Bulleidia',  'Oerskovia']
new_genera_df = balanced_markers[balanced_markers["Genus"].isin(new_genera)]
protein_candidates_markers = analyze_protein_hierarchy_rf(new_genera_df, n_estimators=100, n_top_proteins=5, random_state=42)
protein_candidates_markers[["protein_name", "genera"]]

/tmp/ipykernel_5580/2573382650.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  protein_df['pathway_cluster'] = clusters


,protein_name,genera
0,enoyl-[ reductase [ (ec 1.3.1.9),[Mycoplana]
1,d-aspartate ligase (ec 6.3.1.12),[Oerskovia]
2,"tagatose-bisphosphate aldolase; d-tagatose-1,6...",[Bulleidia]
3,thiosulfate-dehydrogenase (quinone); thiosulfa...,[Oerskovia]
4,cholesterol oxidase (ec 1.1.3.6) 5.3.3.1) (cho...,[Oerskovia]
5,chloramphenicol acetyltransferase (ec 2.3.1.28),[Oerskovia]
6,endo-alpha-n-acetylgalactosaminidase (ec 3.2.1...,[Oerskovia]
7,"2,3-dihydro-2,3-dihydroxybenzoate-dehydrogenas...",[Oerskovia]
8,"2,3-dihydroxybenzoate-amp ligase (ec 2.7.7.58)",[Oerskovia]
9,"selenide, water dikinase; selenophosphate-synt...",[Oxalobacteraceae_unclassified]


# 5. File Integration of Microbiological Data and Physicochemical Data
df_physicochemical comprises the selected features phychem_features and the whole set of parameters if necesary for plotting purposes.
The df_micro is the df of protein-genus features selected from the notebook 7_visual_proteins_ipnyb
micro_usuals is a dictionary with the list of proven bacteria influencing corrosion and could serve as label for plotting purposes
micro_markers is the dictionary with the list of bacteria belonging to the df_micro dataframe.

In [ ]:
data_physicochemical = Path("/home/beatriz/MIC/1_Physicochemical/Data/")
physichem_path = data_physicochemical /"Physicochemical.xlsx"

all_physicochemical = pd.read_excel(physichem_path, sheet_name='all_physicochemical', engine ='openpyxl')
metadata = pd.read_excel(physichem_path, sheet_name='Metadata', engine ='openpyxl')


physichem_features = ['Temperature', 'Type', 'EC_M', 'O2_Eh',
                     'Ox_Fe_Zn', 'Cl_SO4_NO3', 'Na_K','pH_HPO4',
                       'Ca_HCO3_Mg', 'Cu_Al_Mn', 'Ni_Cr_Mo']
physichem_df = all_physicochemical[physichem_features]


In [ ]:
micro_df= pd.read_excel(combined_path, sheet_name='protein_markers',  engine ='openpyxl')
all_physichem = pd.read_excel(combined_path, sheet_name='all_physicochemical', engine ='openpyxl')
genus_site_df = pd.read_excel(combined_path, sheet_name='genus_to_sites', engine ='openpyxl')


ValueError: Worksheet named 'protein_markers' not found

Validation
As a first approximation I will take the first 20 with best score, but I will have to iterate through the list and check if these proteins are really relevant to corrosion states or are also found on the other part of the community the other 800 bacteria and if so what is the numerical diffence, if as I think these are universal proteins and belong infact also to the other part of the community, then we have to discard them and found the ones that really make a difference on the failed systems.

# 5. Proteins Associated with Corrosion from the Literature
In this section a querry to the group balanced markers is done with the aim to find the following proteins which have been associated with corrosion in the literature
all_physiche balanced_markers

In [ ]:

# Define search terms for each protein group - using partial matches rather than just startswith
protein_search_terms = {
    'hydrogenase': ['hydrogenase', '[nife]', '[fefe]', 'fe-only hydrogenase', 'ni-fe hydrogenase'],
    'cytochrome': ['cytochrome c', 'c-type cytochrome', 'cytc'],
    'sulfite_reductase': ['dissimilatory sulfite reductase', 'dsrab', 'dsr', 'sulfite reductase'],
    'metal_resistance': ['cobalt-zinc-cadmium resistance', 'czca', 'metal efflux', 'metal resistance'],
    'quorum_sensing': ['ahl synthase', 'luxi', 'luxr', 'autoinducer synthase', 'quorum sensing'],
    'eps_production': ['glycosyltransferase', 'eps synthase', 'exopolysaccharide']
}

# Function to check if protein name contains any of the search terms
def matches_protein_group(protein_name, search_terms):
    protein_name_lower = protein_name.lower()
    return any(term.lower() in protein_name_lower for term in search_terms)

# Create dictionaries to store results
found_proteins = {category: [] for category in protein_search_terms}

# Search for proteins in each category
for category, terms in protein_search_terms.items():
    # Create condition that checks if protein_name contains any of the search terms
    match_condition = balanced_markers["protein_name"].apply(
        lambda x: matches_protein_group(x, terms)
    )
    
    # Filter the DataFrame
    matched_proteins = balanced_markers[match_condition]
    
    # Store results
    found_proteins[category] = matched_proteins
    for protein_name in matched_proteins["protein_name"].unique():
        # Get the genera for this protein
        protein_rows = matched_proteins[matched_proteins['protein_name'] == protein_name]
        genera_value = protein_rows['Genus'].iloc[0]
        
        # Check if the genera is already a list or a string representation of a list
        if isinstance(genera_value, list):
            genera_list = genera_value
        else:
            # Try to evaluate as a Python list if it's a string representation
            import ast
            try:
                genera_list = ast.literal_eval(genera_value)
            except:
                # If it's just a single string, wrap it in a list
                genera_list = [genera_value]
        
        # Print the protein name and joined genera
        print(f"{protein_name}: {', '.join(genera_list)}")


(r,r)-butanediol-dehydrogenase-meso-butanediol-dehydrogenase: Pseudarthrobacter
enoyl-[ reductase (nadph, si-specific); acyl-acp-dehydrogenase: Gallionella
isocitrate-dehydrogenase [ (ec 1.1.1.42): Gallionella
2-iminoacetate-synthase (ec 4.1.99.19) ([ hydrogenase: Clostridium
(r,r)-butanediol-dehydrogenase; butyleneglycol-dehydrogenase;: Paracoccus
saccharopine-dehydrogenase (nad+, l-lysine-forming: Desulfobulbus
(s,s)-butanediol-dehydrogenase; l-butanediol-dehydrogenase;: Porphyrobacter
meso-butanediol-dehydrogenase-(s,s)-butanediol-dehydrogenase: Variovorax
cytochrome-c3 hydrogenase; h2:ferricytochrome c3 oxido: Desulfobacterium
glycerol-3-phosphate-dehydrogenase (ec 1.1.5.3): Phreatobacter
nadph-dehydrogenase (ec 1.6.99.1): Bacillus
ferredoxin hydrogenase; h2 oxidizing hydrogenase; h2: Hydrogenophaga
2-oxoisovalerate-dehydrogenase subunit alpha (ec 1: Blastomonas
polyvinylalcohol-dehydrogenase (ec 1.1.2.6): Bradyrhizobium
decarboxylase-oxoglutarate-dehydrogenase thiamine pyrophospha

In [ ]:
with pd.ExcelWriter(combined_path, mode="a", engine='openpyxl') , if_sheet_exists="replace" as writer:
    protein_markers.to_excel(writer, sheet_name= "protein_markers", index=True, freeze_panes=(1,0))
    all_physicochemical.to_excel(writer, sheet_name= "all_physicochemical", index=True, freeze_panes=(1,0))
    metadata.to_excel(writer, sheet_name= "Metadata", index=True, freeze_panes=(1,0))
    protein_candidates.to_excel(writer, sheet_name= "candidates", index=True, freeze_panes=(1,0))


SyntaxError: unmatched ')' (3515341291.py, line 1)